In [1]:
import os
import numpy as np
import pandas as pd
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
from sklearn.model_selection import train_test_split
import torch
import random
from tqdm import tqdm
from transformers import BertJapaneseTokenizer, BertForMaskedLM
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification,AdamW, BertConfig, BertModel, BertPreTrainedModel
import pytorch_lightning as pl
#MODEL_NAME='bert-base-cased'
from torch import nn
import MeCab

In [7]:
df_test = pd.read_csv('/home/is/shuntaro-o/dev/persons_move_analysis/data/use_new_model/df_test_adGeoflag.csv')

In [10]:
df_Geoname = df_test[df_test["Geo_Flag"]==1]
df_noGeoname = df_test[df_test["Geo_Flag"]==0]

In [14]:
df_noGeoname

,Unnamed: 0.1,Unnamed: 0,text,created_at,retweets,replies,likes,quote_count,place_id,author_id,...,city_name,object,country,UserPlase_longitude,UserPlase_latitude,UserPlase_sub_longitude,UserPlase_sub_latitude,code,estimate_code,Geo_Flag
0,0,937,サントリー白州蒸溜所ツアー樽貯蔵庫に足を踏み入れた瞬間の香り…忘れられない🥃何度でも訪れたい...,2022-07-12_13:52:40,0.0,0.0,14.0,0,0fc3a3c87d946003,2795144574,...,サントリー白州蒸溜所,poi,日本,138.303687,35.828109,138.303687,35.828109,5338,5436,0
1,1,876,ここ最近購入した目覚まし時計を譲る事となりました。中古品ですが、ナショナルクロックラジオ「簡...,2022-07-26_22:54:46,0.0,0.0,3.0,0,aa52ed3a2a57a535,1246216137559257088,...,奈良 橿原市,city,日本,135.750495,34.463281,135.832225,34.537666,5135,5236,0
2,2,260,ここでも35℃ #35Do https://t.co/Rp42XHmLDS,2022-07-01_10:58:09,0.0,0.0,1.0,0,fcbe3555d7b421d0,122428406,...,愛知 名古屋市 熱田区,city,日本,136.888356,35.108910,136.924501,35.139906,5236,5339,0
3,3,1174,@JirouAruma たぶんそうですよね♪店構えに出るし(笑),2022-07-30_07:53:09,0.0,0.0,2.0,0,61774861ea700565,308818748,...,神奈 藤沢市,city,日本,139.394316,35.297103,139.516992,35.428977,5239,5133,0
6,6,1377,@yooooocho0613 車検通したばっかりなのに、そんな金額なんです？,2022-07-04_12:52:37,0.0,1.0,1.0,0,f5d385fa620f3ba9,521042306,...,新潟 西区,city,日本,138.856177,37.803338,139.024786,37.909360,5638,5439,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65498,65498,109,@1961HAKODATE お疲れ様でしたー🤗暑いね～😮‍💨レシピ通り作ったら美味しかった🎶,2022-07-27_19:59:26,0.0,1.0,1.0,0,725638a075921aa5,2368967174,...,北海道 函館市,city,日本,140.692403,41.710052,141.187479,42.009392,6240,5439,0
65499,65499,1039,@a05071211 おはようSUN🤗,2022-07-25_04:26:35,0.0,1.0,1.0,0,7fa7c66d61604cc5,2801835822,...,広島 広島市 東区,city,日本,132.464266,34.392100,132.575992,34.469456,5132,5439,0
65500,65500,1364,#仲野太賀いい俳優だよね🎵#拾われた男 #松本穂香 #要潤 #薬師丸ひろ子 #草彅剛,2022-07-10_22:53:02,0.0,0.0,1.0,0,22c3d93feadaf1c2,4720869864,...,静岡 浜松市 中区,city,日本,137.685262,34.679768,137.767103,34.780028,5237,5135,0
65501,65501,1403,山中湖は曇り空だったので、そんなに暑くなかったです。ハコニワさんで頂いたのは、新メニューのフ...,2022-07-02_17:54:35,0.0,0.0,6.0,0,04137ba8263d0a7a,253046034,...,神奈 南足柄市,city,日本,139.004641,35.271780,139.137657,35.352280,5239,5338,0


In [15]:
df_noGeoname[df_noGeoname['code']==df_noGeoname['estimate_code']]

,Unnamed: 0.1,Unnamed: 0,text,created_at,retweets,replies,likes,quote_count,place_id,author_id,...,city_name,object,country,UserPlase_longitude,UserPlase_latitude,UserPlase_sub_longitude,UserPlase_sub_latitude,code,estimate_code,Geo_Flag
21,21,217,あんたらはなーんもせんやんけ。 https://t.co/AcCGjscukB,2022-07-28_03:48:00,0.0,1.0,6.0,0,2ea0f6f3519427cb,1257910174540394496,...,京都 久御山町,city,日本,135.713101,34.858133,135.833847,34.911655,5235,5235,0
42,42,185,@lucas49fct 午前中はお客さんも少なめなので狙い目ではあります,2022-07-23_17:59:19,0.0,1.0,1.0,0,76c498282a5a7997,2233368990,...,東京 足立区,city,日本,139.735751,35.738659,139.858685,35.817519,5339,5339,0
46,46,446,@010CHO3 UMAJOはいたんですか？しかも女性⁉️,2022-07-18_00:54:13,0.0,1.0,0.0,0,8faa6bda62c22c5d,417770098,...,大阪 池田市,city,日本,135.415794,34.787565,135.460832,34.878023,5235,5235,0
51,51,254,@ljWemZ0KRSSNLyn 野外でもブルマが大活躍‼️😗素晴らしい💞 ブルマは万能‼️😗,2022-07-12_21:58:48,0.0,0.0,1.0,0,bf94246d88e50b65,1080183375141363712,...,大阪 大東市,city,日本,135.590437,34.695705,135.675169,34.732090,5235,5235,0
67,67,1055,@oB2UH9VporRF3XL いや、単に老けてます！,2022-07-27_20:55:51,0.0,1.0,1.0,0,1e3c41162a8500a2,1382157784012115968,...,山口 下関市,city,日本,130.774893,33.911121,131.172123,34.373845,5030,5030,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65463,65463,1089,@aixSgQO4RiQUjkS おはようございます。😊今週もマイペースで参りましょう。👍,2022-07-11_06:51:52,0.0,1.0,3.0,0,d0ed06276bdaf968,1327634654,...,香川 高松市,city,日本,133.920998,34.111199,134.176491,34.433944,5133,5133,0
65464,65464,769,体がうずうずするセトリ🥹 https://t.co/yrQwAIK3vf,2022-07-09_13:56:35,0.0,1.0,8.0,0,108ee55f79545001,1258251780229488640,...,Bayside Live Hall,poi,日本,130.399342,33.603398,130.399342,33.603398,5030,5030,0
65474,65474,124,真夏のゴルフで暑かったですが午後からは風が少し出てきてよかった。#信州伊那国際ゴルフクラブ ...,2022-07-02_15:59:25,0.0,0.0,0.0,0,eb7a49f03b5d8f9c,98754590,...,長野 南箕輪村,city,日本,137.847166,35.850043,137.993522,35.925466,5337,5337,0
65489,65489,1301,変身と技名のセリフ量が多いよ〜#nitiasa #precure,2022-07-10_08:53:37,0.0,0.0,0.0,0,725638a075921aa5,393421233,...,北海道 函館市,city,日本,140.692403,41.710052,141.187479,42.009392,6240,6240,0
